## Create and Destroy a Container Registry in Azure using Terraform

- An Azure Container Registry is used to store Images (similar to DockerHub).
  - Since an Azure Container Registry is an Azure resource, it must be placed in an Azure Resource Group.
- This Terraform Project consists of the Terraform files listed below:

In [1]:
#!dir *.tf # use this on Windows
!ls *.tf

container-registry.tf  providers.tf  resource-group.tf	variables.tf


## Terraform providers

- We are using the same Terraform proviers as before (i.e. the `azurerm` provider for Azure).

In [2]:
#!type providers.tf # use this on Windows
!cat providers.tf

# Initialises Terraform providers and sets their version numbers.

terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "3.90.0"
    }
  }

  required_version = "~> 1.7.0"
}

provider "azurerm" {
  features {}
}

## Terraform variables

- We are using the same Terraform variables as before.

**Note! Make sure you change the value for the variable `app_name` to something unique!**

In [3]:
#!type variables.tf # use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.

variable app_name {
  default = "tsfn14g00"
}

variable location {
  default = "westeurope"
}

## Azure Resource Group

- We are using the same Azure Resoure Group as before.

In [4]:
#!type resource-group.tf # use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}

## Let's view the contents of the file `container-registry.tf`

- Here we are defining an Azure Resource Group
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_container_registry`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `container_registry` is the name of the Azure resource (i.e. an Azure Container Registry defined in the `azurerm` provider/plugin).
  - The first Argument sets the Azure Container Registry's name
    - `name` is the argument's name
    - Its value is retrieved from the Terraform variable `app_name` (defined in the file `variables.tf`).
  - The second Argument sets the Azure Resource Group in which the Azure Container Registry will be created
    - `resource_group_name` is the argument's name
    - Its value is retrieved from the Terraform Expression `azurerm_resource_group.main.name`.
      - The `azurerm_resource_group.main` Block is defined in `resource-group.tf` as `resource "azurerm_resource_group" "main"`.
      - In this Block, there is an Argument with a name of `name` who's value is defined as `var.app_name`.
      - This is the value that is assigned to `resource_group_name`.
  - The third Argument sets the Azure Container Registry's Location
    - `location` is the argument's name
    - Its value is retrieved from the Terraform variable `location` (defined in the file `variables.tf`).
  - The fourth Argument enables the Azure Container Registry's Amin Account
    - `locaadmin_enabledtion` is the argument's name
    - Its value is set to `true`.
    - For more information about the Azure Container Registry's Admin accoun, see:
      - https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli
  - The fifth Argument sets the Azure Container Registry's service tier (also known as SKU).
    - `sku` is the argument's name
    - Its value is set to `"Basic"`.
    - For more information about Azure Container Registry SKUs, see:
      - https://learn.microsoft.com/en-us/azure/container-registry/container-registry-skus.

In [5]:
#!type container-registry.tf # use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".


resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}

## Initialize Terraform

- The Terraform CLI command `terraform init` initializes the Terraform Project.

In [6]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## Terraform Apply

- The Terraform CLI command `terraform apply -auto-approve` applies (creates/updates) the resources defined in the Terraform project.

In [7]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the Resource Group defined in the Terraform project has been created.

In [8]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Resources in Resource Group `tsfn14g00`

- The Azure CLI command `az resource list -n tsfn14g00 -o table` lists resources in Resource Group `tsfn14g00`.
- We see that the Resource Group contains the Container Registry.

In [9]:
!az resource list -n tsfn14g00 -o table

Name       ResourceGroup    Location    Type                                    Status
---------  ---------------  ----------  --------------------------------------  --------
tsfn14g00  tsfn14g00        westeurope  Microsoft.ContainerRegistry/registries


## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been created.

In [10]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-09T22:52:15Z  True


## List Repositories in Container Registry `tsfn14g00 `

- The Azure CLI command `az acr repository list -n tsfn14g00 --top 10 -o table` lists Repositories in a Azure Container Registry `tsfn14g00`.
  - The `-n` option is manditory and specifies the `NAME` of the Container Registry.
  - The `--top 10` limits the list to the first 10 Repositories (remove to see all Repositories).
- We see that Container Registry `tsfn14g00` doesn't contain any Repositories.

In [11]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Show Information about Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr show -n tsfn14g00 -o table` shows information about Container Registry `tsfn14g00`.
- It shows the Container Registry's `LOGIN SERVER` which is the URL to your Container Registry on Azure.
  - **This is the URL you would use to upload Docker Images to the Azure Container Registry.**

In [12]:
!az acr show -n tsfn14g00 -o table

# Let's store the LOGIN SERVER in a Python variable so we can use it later in this notebook
CONTAINER_REGISTRY_LOGIN_SERVER=!az acr show -n tsfn14g00 --query loginServer -o tsv
CONTAINER_REGISTRY_LOGIN_SERVER=CONTAINER_REGISTRY_LOGIN_SERVER[0]

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-09T22:52:15Z  True


## Show Credentials for Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr credential show -n tsfn14g00 -o table` shows credentials about Container Registry `tsfn14g00`.
- It shows the Container Registry's `USERNAME` and  `PASSWORD` to use to authenticate with your Azure Container Registry.
  - **This is the USERNAME and PASSWORD you would use to login to Docker to upload Images to the Azure Container Registry.**

In [13]:
!az acr credential show -n tsfn14g00 -o table

# Let's store the USERNAME and PASSWORD in Python variables so we can use them later in this notebook
CONTAINER_REGISTRY_USERNAME=!az acr credential show -n tsfn14g00 --query username -o tsv
CONTAINER_REGISTRY_USERNAME=CONTAINER_REGISTRY_USERNAME[0]
CONTAINER_REGISTRY_PASSWORD=!az acr credential show -n tsfn14g00 --query passwords[0].value -o tsv
CONTAINER_REGISTRY_PASSWORD=CONTAINER_REGISTRY_PASSWORD[0]

USERNAME    PASSWORD                                              PASSWORD2
----------  ----------------------------------------------------  ----------------------------------------------------
tsfn14g00   J1E8NJMcMh9HeVq9RyQxV4lH+/2A3mUB5Rn8L+FyqX+ACRDXHkVM  tzkfV4ihviN3Anq5hmBIHAscd5/ZogCBtyPRlr60X7+ACRC3Fy90


## Login to Azure Container Registry via Docker

- Replace the variables below with your `LOGIN_SERVER`, `USERNAME` and `PASSWORD`.

In [15]:
!docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME -p $CONTAINER_REGISTRY_PASSWORD

# In Ubuntu with environment variables CONTAINER_REGISTRY_LOGIN_SERVER, CONTAINER_REGISTRY_USERNAME and CONTAINER_REGISTRY_USERNAME
#!echo $PASSWORD | docker login $LOGIN_SERVER -u $USERNAME --password-stdin  > /dev/null 2>&1

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


## Let's view the code in `video-streaming/src/index.js`

- The file contains code for creating a simple Node.js application, including the steps:
  1. Require the NPM package "express".
  2. Read the environment variable PORT to use as the Express web server's listen port.
  3. Creates an instance of an Express web server.
  4. Define a GET route for the "/video" path.
     1. Read information about the video file "videos/SampleVideo_1280x720_1mb.mp4".
     2. Write the HTTP response header including information about the file size in bytes and content type (video/mp4).
     3. Stream the file contents to the HTTP GET response.
  5. Start the Express web server listening on the port stored in environment variable PORT.
- As we see, this code produces a simple microservice that streams a video file from the file system to an HTTP GET response.

In [16]:
cat video-streaming/src/index.js

const express = require("express");
const fs = require("fs");

if (!process.env.PORT) {
    throw new Error("Please specify the port number for the HTTP server with the environment variable PORT.");
}

const PORT = process.env.PORT;

//
// Application entry point.
//
async function main() {

    const app = express();

    app.get("/video", async (req, res) => { // Route for streaming video.
        
        const videoPath = "./videos/SampleVideo_1280x720_1mb.mp4";
        const stats = await fs.promises.stat(videoPath);
    
        res.writeHead(200, {
            "Content-Length": stats.size,
            "Content-Type": "video/mp4",
        });

        fs.createReadStream(videoPath).pipe(res);
    });

    app.listen(PORT, () => {
        console.log("Microservice online.");
    });
}

main()
    .catch(err => {
        console.error("Microservice failed to start.");
        console.error(err && err.stack || err);
    });

## Initialize the Node.js project

- Let's initialise the Node.js project for the microservice by:
  - Running `npm init`.
  - Running `npm pkg` to configure `package.json`.
  - Running `npm install` to install the required NPM packages.

In [17]:
%%system
#rm -rf ./video-streaming/node_modules ./video-streaming/package.json ./video-streaming/package-lock.json
cd video-streaming
npm init -y
npm pkg set 'main'='./src/index.js'
npm pkg set 'scripts.start'='node ./src/index.js'
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js'
npm pkg delete 'scripts.test'
npm install --silent --save express@5.0.0-beta.1
npm install --silent --save-dev nodemon@3.0.3
cd ..

['Wrote to /home/patrick/projects/tsfn14/tsfn14/03_Azure_and_Terraform/03_container_registry/video-streaming/package.json:',
 '',
 '{',
 '  "name": "video-streaming",',
 '  "version": "1.0.0",',
 '  "description": "",',
 '  "main": "index.js",',
 '  "scripts": {',
 '    "test": "echo \\"Error: no test specified\\" && exit 1"',
 '  },',
 '  "keywords": [],',
 '  "author": "",',
 '  "license": "ISC"',
 '}',
 '',
 '']

## Let's look at the Docker file used to build an image of the Node.js project

- Our image is based on the slimmed-down `alpine` version of `node:19.9.0`.
- The working directory in the image is `/usr/src/app`.
- The files `package.json` and `package-lock.json` are copied from the host computer to the image's working directory.
- The command `npm ci --omit-dev` is run in the image's working directory which installs non-dev NPM packages from `package.json`.
- The file `index.js` is copied from the host computer's `src` folder to the image's working directory's `src` folder.
- The file `SampleVideo_1280x720_1mb.mp4` is copied from the host computer's `videos` folder to the image's working directory's `videos` folder.
- The command `npm start` will be run in the working directory when an image container is started.

In [18]:
#!type video-streaming/Dockerfile # use this on Windows
!cat video-streaming/Dockerfile

FROM node:19.9.0-alpine

WORKDIR /usr/src/app
COPY package*.json ./
RUN npm ci --omit=dev
COPY ./src ./src
COPY ./videos ./videos

CMD npm start

## Build and Push a Docker Image to Azure Container Registry

- Here we are building an image of the Node.js application (microservice).
- We are tagging the image as `tsfn14g00.azurecr.io/video-streaming:1,`where:
  - `tsfn14g00.azurecr.io` is the URL (LOGIN SERVER) to our Container Registry.
  - `video-streaming` is the name of our image (repository).
  - `1` is the version of the image (tag).
- Then the image is pushed to the Azure Container Registry.
- Finally, the local image is removed from the host computer.

In [19]:
%%system

# Build Docker image with Nodejs Application
docker build -q -t {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1 -f ./video-streaming/Dockerfile ./video-streaming

# Push Docker Image to Azure Container Registry
docker push {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1

# Clean up
docker rmi {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1
docker images {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1

['failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory',
 '',
 'DEPRECATED: The legacy builder is deprecated and will be removed in a future release.',
 '            Install the buildx component to build images with BuildKit:',
 '            https://docs.docker.com/go/buildx/',
 '',
 'sha256:73789e685819ffc37dbe32f7286ad311c23747c9a1c6eb446dc63d7d2b6472ee',
 'The push refers to repository [tsfn14g00.azurecr.io/video-streaming]',
 '1e40cca48f0d: Preparing',
 'cc739988f1f4: Preparing',
 '294e44ad6e13: Preparing',
 '64cbb0cf5f3d: Preparing',
 'efab73490ce2: Preparing',
 'e394b5a78515: Preparing',
 '9b2d5af7f709: Preparing',
 'b240fe81adaa: Preparing',
 'bb01bd7e32b5: Preparing',
 'e394b5a78515: Waiting',
 'bb01bd7e32b5: Waiting',
 '9b2d5af7f709: Waiting',
 'efab73490ce2: Pushed',
 'cc739988f1f4: Pushed',
 '64cbb0cf5f3d: Pushed',
 '294e44ad6e13: Pushed',
 '1e40cca48f0d: Pushed',
 'e394b5a78515: Pushed',
 '9b2d5af7f709: Pushed',
 'bb

## List Repositories in Container Registry `tsfn14g00 `

- We see that the Repository `video-streaming` has been created in Container Registry `tsfn14g00`.

In [20]:
!az acr repository list -n tsfn14g00 --top 10 -o table

Result
---------------
video-streaming


## Show Information about Repository `video-streaming`

- The Azure CLI command `az acr repository show -n tsfn14g00 --repository video-streaming -o table`
  - Shows information about Repository `video-streaming` in Container Registry `tsfn14g00`.
    - It contains images named `video-streaming` (ImageName).
    - It has a tag count of `1` (TagCount), i.e. currently there is only one tag for the `video-streaming` image.

In [21]:
!az acr repository show -n tsfn14g00 --repository video-streaming -o table

CreatedTime                  ImageName        LastUpdateTime                ManifestCount    Registry              TagCount
---------------------------  ---------------  ----------------------------  ---------------  --------------------  ----------
2024-02-09T23:14:19.067442Z  video-streaming  2024-02-09T23:14:19.1462558Z  1                tsfn14g00.azurecr.io  1


## List Tags in Repository `video-streaming`

- The Azure CLI command `az acr repository show-tags -n tsfn14g00 --repository video-streaming --top 10 -o table`:
  - Lists the tags in Repository `video-streaming` in Container Registry `tsfn14g00`.
    - Currently there is only one tag.
    - The tag has the value `1`.

In [22]:
!az acr repository show-tags -n tsfn14g00 --repository video-streaming --top 10 -o table

Result
--------
1


## Show Information about Image `video-streaming:1`

- The Azure CLI command `az acr repository show -n tsfn14g00 --image video-streaming:1 -o table`:
  - Shows information about image `video-streaming:1` in Container Registry `tsfn14g00`.
    - The information includes the Digest for the image.

In [23]:
!az acr repository show -n tsfn14g00 --image video-streaming:1 -o table

CreatedTime                   Digest                                                                   LastUpdateTime                Name    Signed
----------------------------  -----------------------------------------------------------------------  ----------------------------  ------  --------
2024-02-09T23:14:19.1754364Z  sha256:b6799df61b875afeb217a5c610fee236d7ebc80c806051905641000c7cc70b61  2024-02-09T23:14:19.1754364Z  1       False


## Show Azure Container Registry Usage

- The command `az acr show-usage -n tsfn14g00 -o table` shows the usage of Container Registry `tsfn14g00`.
  - We can see the maximum number of allowed bytes in the `LIMIT` column (first row).
  - We can see the current number of bytes in the `CURRENT VALUE` column (first row).

In [24]:
!az acr show-usage -n tsfn14g00 -o table

NAME       LIMIT        CURRENT VALUE    UNIT
---------  -----------  ---------------  ------
Size       10737418240  56544649         Bytes
Webhooks   2            0                Count
ScopeMaps  100          0                Count
Tokens     100          0                Count


## Pull and Run an Image from the Azure Container Registry

- The image `video-streaming:1` will be pulled from the Azure Container Registry to the host computer.
- Then a Container is created from the Image, where:
  - The host computer's port 3000 is mapped to the container's port 3000.
  - An environment variable PORT is created in the container with the value 3000.

In [25]:
%%system

docker run --name video-streaming -d -p 3000:3000 -e PORT=3000 {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1
docker ps
docker logs video-streaming
#docker exec -it video-streaming /bin/sh

["Unable to find image 'tsfn14g00.azurecr.io/video-streaming:1' locally",
 '1: Pulling from video-streaming',
 '8a49fdb3b6a5: Already exists',
 '1197750296b3: Already exists',
 'f352bc07f19b: Already exists',
 '47be83a79857: Already exists',
 '7c392e84349e: Pulling fs layer',
 '7a544e00f2c1: Pulling fs layer',
 '2983b1db984a: Pulling fs layer',
 '3ec43100824d: Pulling fs layer',
 '9fc130cc4c7b: Pulling fs layer',
 '9fc130cc4c7b: Waiting',
 '3ec43100824d: Waiting',
 '7c392e84349e: Verifying Checksum',
 '7c392e84349e: Download complete',
 '7a544e00f2c1: Verifying Checksum',
 '7a544e00f2c1: Download complete',
 '7c392e84349e: Pull complete',
 '7a544e00f2c1: Pull complete',
 '2983b1db984a: Verifying Checksum',
 '2983b1db984a: Download complete',
 '3ec43100824d: Download complete',
 '9fc130cc4c7b: Verifying Checksum',
 '9fc130cc4c7b: Download complete',
 '2983b1db984a: Pull complete',
 '3ec43100824d: Pull complete',
 '9fc130cc4c7b: Pull complete',
 'Digest: sha256:b6799df61b875afeb217a5c610

## Access the Microservice from a Web Browser

- Open a web browser and enter the URL `http://localhost:3000/video`
  - This will send an HTTP GET request to the microservice's GET route for the "/video" path.
  - The video stored in the microservice's container will be streamed to the web browser.

In [26]:
#!firefox http://localhost:3000/video

## Stop and Remove the Docker Container and Image from your computer

- The Container `video-streaming` is stopped and removed from the host computer.
- Then the Image `video-streaming:1` is removed from the host computer.

In [27]:
%%system

docker stop video-streaming
docker rm video-streaming
docker ps -a
docker rmi {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1
docker images {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1

['video-streaming',
 'video-streaming',
 'CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES',
 'Untagged: tsfn14g00.azurecr.io/video-streaming:1',
 'Untagged: tsfn14g00.azurecr.io/video-streaming@sha256:b6799df61b875afeb217a5c610fee236d7ebc80c806051905641000c7cc70b61',
 'Deleted: sha256:73789e685819ffc37dbe32f7286ad311c23747c9a1c6eb446dc63d7d2b6472ee',
 'Deleted: sha256:a377240bf99c8c856959aa17459440cc87420125d249a0bf18501d0cab364522',
 'Deleted: sha256:65ba23a159f50dc7f8970b80a04e446d73da6d84bca2e52558376bb0a27df7d1',
 'Deleted: sha256:b74831dfb96b4f02b72aaa80db843698e5924252172cc9609b9b1aecf13d35bc',
 'Deleted: sha256:214cc8a32e48279af14ac92f495d7002797fd92445989adb4dd500d642d69405',
 'Deleted: sha256:cb3a616826bab0dda9186eeb9cd167cbe6dc9ff05962a8d5791bd067043b48b9',
 'REPOSITORY   TAG       IMAGE ID   CREATED   SIZE']

## Logout from the Azure Container Registry via Docker

In [30]:
!docker logout $CONTAINER_REGISTRY_LOGIN_SERVER

Removing login credentials for tsfn14g00.azurecr.io


## Delete Repository `video-streaming` from Azure Container Registry

- The Azure CLI command `az acr repository delete -y -n tsfn14g00 --repository video-streaming -o table`
  - Deletes the Repository `video-streaming` in Container Registry `tsfn14g00`.
  - Use this command to delete all images in Repository `video-streaming`.
- The Azure CLI command `az acr repository delete -y -n tsfn14g00 --image video-streaming:1 -table`
  - Deletes the Image `video-streaming:1` in Container Registry `tsfn14g00`.
  - Use this command to delete one image from Repository `video-streaming`.

In [29]:
#!az acr repository delete -y -n tsfn14g00 --image video-streaming:1 -table
!az acr repository delete -y -n tsfn14g00 --repository video-streaming -o table

## List Repositories in Container Registry `tsfn14g00 `

- We see that the Repository `video-streaming` has been deleted from Container Registry `tsfn14g00`.

In [31]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Terraform Destroy

- The Terraform CLI command `terraform destroy -auto-approve` destroys the resources defined in the Terraform project.

In [32]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_container_registry.main will be destroyed
  - resource "azurerm_container_registry" "main" {
      - admin_enabled                 = true -> null
      - admin_password                = (sensitive value) -> null
      - admin_username                = "tsfn14g00" -> null
      - anonymous_pull_enabled        = false -> null
      - data_endpoint_enabled         = false -> null
      - encryption                    = [
          - {
              - enabled    

## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been destroyed.

In [33]:
!az acr list -o table

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the Resource Group defined in the Terraform project has been destroyed.

In [34]:
!az group list -o table